In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
IN_COLAB = 'google.colab' in str(get_ipython())
if IN_COLAB:
  !pip install git+https://github.com/pete88b/nbdev_colab_helper.git
  from nbdev_colab_helper.core import *
  project_name = 'nextai'
  init_notebook(project_name)

 # auto_agument

> Implements Google AutoAugment augmentations.
 

In [ ]:
#default_exp auto_augment

In [ ]:
#hide
!pip install fastai --upgrade --quiet

     |████████████████████████████████| 358kB 2.7MB/s 


In [ ]:
%nbdev_export
from torch import tensor, Tensor
import torch

In [ ]:
%nbdev_export
# Automatically sets for GPU or CPU environments
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 

#### Helper functions

In [ ]:
%nbdev_export
# Convert FASTAI image basis. (-1,-1) to (1,1)  to PIL image basis (0,0) to (1,1)    
def fastai2pil_basis(b) :  return ((b + 1.)).div(2.)      # b - Bounding box(s)     

In [ ]:
%nbdev_export
# Convert PIL image basis (0,0) to (1,1) to FASTAI image basis. (-1,-1) to (1,1) 
def pil2fastai_basis(b):  return (b * 2.).float() - 1.    # b - Bounding box(s)   

In [ ]:
%nbdev_export
def flip_horizontal(bboxes): 
  ''' 
    Flips a bounding box tensor along the vertical axis 
    Input:    bboxes - 2-d tensor containing bounding boxes
    Output:   Bounding boxes flipped along the vertical axis
  '''
  bboxes[:,[1,3]] = torch.flip(bboxes[:,[1,3]], [1])      # Swap the (x) columns: 1, and 3
  bboxes[:,[1, 3]] *= -1;                                 #   Flip the sign of each of these columns
  return bboxes

In [ ]:
%nbdev_export
def swap_xy_coords (bboxes):   
  ''' swap yx coordinate sequences in bounding boxes into xy sequences, and viceversa 
      Input:    bboxes - 2-d tensor containing bounding boxes
      Output:   Bounding boxes flipped with swapped coordinate, xy --> yx
  '''
  bboxes[:,[0,1]] = torch.flip(bboxes[:,[0,1]], [1]) 
  bboxes[:,[2,3]] = torch.flip(bboxes[:,[2,3]], [1])
  return bboxes


In [ ]:
%nbdev_export
def rotate_bb(bb, rads): 
    ''' Rotate a bounding box (x1,y1,x2,y2) by an angle 
        Input:  bb -   bounding box
                rads - rotation angle in radians 
        '''           
    M = torch.tensor([                         # Rotation Matrix
           [math.cos(rads), -math.sin(rads)],         
           [math.sin(rads),  math.cos(rads)]
           ] ).to(device)           
    return torch.mm(M, bb.to(device))           #   


#### Bounding Box transformations

In [ ]:
%nbdev_export
# SHEAR-HORIZONTALLY BOUNDING BOXES
def shear_x_bboxes (bboxes:Tensor, factor:float, y_first=True):
  ''' 
    Shear horizontally a tensor of bounding boxes
    Input:      
          bboxes  -      2-d tensor of bounding boxes associated with the image
          factor  -      Factor by which the image in sheared in the horizontal direction
          y_first -      Input coordinates in the format y1x1y2x2
          TODO: change this
    Output:
          bboxes -       Sheared bounding boxes
  '''
  if not y_first: swap_xy_coords(bboxes)                                # swap yx sequence for xy sequence
  m = bboxes[(bboxes == 0.).all(1)]                                     # Retain the all-zero rows
  bboxes = bboxes[~(bboxes == 0.).all(1)]                               # Retain the non all-zero rows
  mag = factor                                                          # If the factor is negative, flip the boxes about the (0,0) center
  if factor <= 0 : mag = -factor; bboxes = flip_horizontal(bboxes)      # so it can be sheared correctly (in the positive orientation)
  bboxes = fastai2pil_basis(bboxes)                                     # Convert to PIL image basis (0,0) to (1,1)       
  bboxes[:,[1,3]] = bboxes[:,[1,3]] + bboxes[:,[0,2]]  * mag            # Shear in the horizontal direction (to the right)
  bboxes = pil2fastai_basis(bboxes)                                     # Convert to FASTAI image basis. Top-left (-1,-1) to Bottom-right (1,1)
  if factor <= 0 : bboxes = flip_horizontal(bboxes)                     # If factor is negative, restore the boxes to the original orientation
  bboxes = torch.clamp(bboxes, -1, 1)                                   # Clamp coordinates to [-1, 1]
  bboxes = torch.cat([m, bboxes], dim=0)                                # Graft the all-zero rows back to the bounding box array    
  if not y_first: swap_xy_coords(bboxes)                                # restore xy sequence
  return bboxes


In [ ]:
%nbdev_export
# ROTATE BOUNDING BOXES
def rotate_bboxes(bboxes:Tensor, degrees:float, y_first=True):
  ''' 
    Rotate bounding boxes (in sync with a rotated image)
    Input:
        bboxes :       2-d tensor of bounding boxes in the format x1,y1,x2,y2
        degrees :      Angle in degrees to rotate the box
        y_first -      Input coordinates in the format y1x1y2x2
        TODO: change this  
    Output:
        bboxes          2-d tensor of rotated bounding boxes
  '''
  rads = math.radians(degrees)                                          # Convert degrees to radians   
  if not y_first: swap_xy_coords(bboxes)                                # swap yx sequence for xy sequence
  m = bboxes[(bboxes == 0.).all(1)]                                     # Retain the all-zero rows of the bounding box
  bboxes = bboxes[~(bboxes == 0.).all(1)]                               # Retain the non all-zero rows of the bounding box
  lgt = abs(bboxes[:,[1]] - bboxes[:,[3]])                              # Calculate the length of the box in the x axis
  mag = rads                                                            # If degrees is negative, flip the boxes about the (0,0) center
  if degrees <= 0 : mag = -rads; bboxes = flip_horizontal(bboxes)       # so it can be rotated correctly (in the positive orientation)
  bboxes = bboxes.reshape(-1,2).transpose(1,0)                          # Put tensor into a (n x 2) vertical array
  bboxes = rotate_bb(bboxes, mag).transpose(0,1).reshape(-1,4)          # Rotate bounding box and restore coordinates to fastai image basis 

  bboxes [:,[0]] = bboxes [:,[0]] - (lgt)*math.sin(mag)                 # Calculate the delta-lenght to add and substract to 
  bboxes [:,[2]] = bboxes [:,[2]] + (lgt)*math.sin(mag)                 #   the y coordinates to compensate for the rotation
  if degrees <= 0 : bboxes = flip_horizontal(bboxes)                    # If degrees is negative, restore the boxes to the original orientation
  bboxes = torch.clamp(bboxes, -1, 1)                                   # Clamp coordinates to [-1, 1]

  bboxes = torch.cat([m, bboxes], dim=0)                                # Graft the all-zero rows back to the bounding box array
  if  not y_first: swap_xy_coords(bboxes)                               # Restore xy sequence
  return bboxes

#### Data Augmentation algorithms

In [ ]:
from nbdev import *
show_doc(rotate_bboxes)
show_doc(shear_x_bboxes)
show_doc(rotate_bb)
show_doc(swap_xy_coords)
show_doc(flip_horizontal)
show_doc(rotate_bboxes)
show_doc(rotate_bboxes)
show_doc(rotate_bboxes)

<h4 id="rotate_bboxes" class="doc_header"><code>rotate_bboxes</code><a href="__main__.py#L3" class="source_link" style="float:right">[source]</a></h4>

> <code>rotate_bboxes</code>(**`bboxes`**:`Tensor`, **`degrees`**:`float`, **`y_first`**=*`True`*)

Rotate bounding boxes (in sync with a rotated image)
Input:
    bboxes :       2-d tensor of bounding boxes in the format x1,y1,x2,y2
    degrees :      Angle in degrees to rotate the box
    y_first -      Input coordinates in the format y1x1y2x2
    TODO: change this  
Output:
    bboxes          2-d tensor of rotated bounding boxes

<h4 id="shear_x_bboxes" class="doc_header"><code>shear_x_bboxes</code><a href="__main__.py#L3" class="source_link" style="float:right">[source]</a></h4>

> <code>shear_x_bboxes</code>(**`bboxes`**:`Tensor`, **`factor`**:`float`, **`y_first`**=*`True`*)

Shear horizontally a tensor of bounding boxes
Input:      
      bboxes  -      2-d tensor of bounding boxes associated with the image
      factor  -      Factor by which the image in sheared in the horizontal direction
      y_first -      Input coordinates in the format y1x1y2x2
      TODO: change this
Output:
      bboxes -       Sheared bounding boxes

<h4 id="rotate_bb" class="doc_header"><code>rotate_bb</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>rotate_bb</code>(**`bb`**, **`rads`**)

Rotate a bounding box (x1,y1,x2,y2) by an angle 
Input:  bb -   bounding box
        rads - rotation angle in radians 

<h4 id="swap_xy_coords" class="doc_header"><code>swap_xy_coords</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>swap_xy_coords</code>(**`bboxes`**)

swap yx coordinate sequences in bounding boxes into xy sequences, and viceversa 
Input:    bboxes - 2-d tensor containing bounding boxes
Output:   Bounding boxes flipped with swapped coordinate, xy --> yx

<h4 id="flip_horizontal" class="doc_header"><code>flip_horizontal</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>flip_horizontal</code>(**`bboxes`**)

Flips a bounding box tensor along the vertical axis 
Input:    bboxes - 2-d tensor containing bounding boxes
Output:   Bounding boxes flipped along the vertical axis

<h4 id="rotate_bboxes" class="doc_header"><code>rotate_bboxes</code><a href="__main__.py#L3" class="source_link" style="float:right">[source]</a></h4>

> <code>rotate_bboxes</code>(**`bboxes`**:`Tensor`, **`degrees`**:`float`, **`y_first`**=*`True`*)

Rotate bounding boxes (in sync with a rotated image)
Input:
    bboxes :       2-d tensor of bounding boxes in the format x1,y1,x2,y2
    degrees :      Angle in degrees to rotate the box
    y_first -      Input coordinates in the format y1x1y2x2
    TODO: change this  
Output:
    bboxes          2-d tensor of rotated bounding boxes

<h4 id="rotate_bboxes" class="doc_header"><code>rotate_bboxes</code><a href="__main__.py#L3" class="source_link" style="float:right">[source]</a></h4>

> <code>rotate_bboxes</code>(**`bboxes`**:`Tensor`, **`degrees`**:`float`, **`y_first`**=*`True`*)

Rotate bounding boxes (in sync with a rotated image)
Input:
    bboxes :       2-d tensor of bounding boxes in the format x1,y1,x2,y2
    degrees :      Angle in degrees to rotate the box
    y_first -      Input coordinates in the format y1x1y2x2
    TODO: change this  
Output:
    bboxes          2-d tensor of rotated bounding boxes

<h4 id="rotate_bboxes" class="doc_header"><code>rotate_bboxes</code><a href="__main__.py#L3" class="source_link" style="float:right">[source]</a></h4>

> <code>rotate_bboxes</code>(**`bboxes`**:`Tensor`, **`degrees`**:`float`, **`y_first`**=*`True`*)

Rotate bounding boxes (in sync with a rotated image)
Input:
    bboxes :       2-d tensor of bounding boxes in the format x1,y1,x2,y2
    degrees :      Angle in degrees to rotate the box
    y_first -      Input coordinates in the format y1x1y2x2
    TODO: change this  
Output:
    bboxes          2-d tensor of rotated bounding boxes